# Web Scrapping & News Articles Clustering

# Getting the data from URL

In [1]:
import requests
from bs4 import BeautifulSoup
url = "https://www.opindia.com/latest-news/"
data = requests.get(url)

In [2]:
soup = BeautifulSoup(data.content)

# Fetching multiple the URLs 

In [3]:
def fet_article(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.content)
    articles = []
    for i in soup.find_all("h3",class_=["entry-title td-module-title"]):
        art = i.find("a")["title"]
        articles.append(art)
    return articles

In [4]:
urls_list = []
for i in range(1,21,1):
    a = "https://www.opindia.com/latest-news/page/" + str(i) +"/"
    urls_list.append(a)

# Fetching articles from the URLs

In [5]:
articles =[]
for i in urls_list:
    a = fet_article(i)
    articles.extend(a)

In [6]:
p_art = []
from re import sub
for i in articles:
    q = sub("[^0-9a-zA-Z ]","",i.lower())
    from nltk.stem import PorterStemmer
    sentc = ""
    for j in q.split(" "):
        ps = PorterStemmer()
        sentc = sentc + " " + ps.stem(j)
    p_art.append(sentc)
  

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
A = tf.fit_transform(p_art).toarray()

# Clustering

In [8]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5)
cl_res = km.fit(A)

In [9]:
cl_res.labels_

array([0, 4, 3, ..., 0, 0, 0])

In [10]:
import pandas as pd
Q = pd.DataFrame(p_art,columns=["Article"])
Q['Cluster']=cl_res.labels_

In [11]:
E = {
0 : "geopolitics",
1 : "politics",
2 : "religion",
3 : "sports",
4 : "entertainment"
}

In [12]:
R = []
for i in Q.Cluster:
    R.append(E[i])

Q['category'] = R

In [13]:
Q

,Article,Cluster,category
0,ladki hoon lad sakti hoon women congress lead...,0,geopolitics
1,tmc vp yashwant sinha delet old abus tweet af...,4,entertainment
2,bjp like to send it 106 maharashtra mla to ah...,3,sports
3,guru rahman the educ under scanner for allege...,0,geopolitics
4,maharashtra mlc elect what exactli happen and...,2,religion
...,...,...,...
2075,churu rajasthan bride marri anoth man as groo...,0,geopolitics
2076,did you know indranarayana swami templ built ...,0,geopolitics
2077,racism aliv and well indian origin student su...,0,geopolitics
2078,kejriw minor appeas delhi govt ask privat sch...,0,geopolitics


# Fetching similar News articles (of same category)

In [14]:
Q[Q["category"]=="politics"]

,Article,Cluster,category
16,wa manmohan singh lobbi for an individu dure ...,1,politics
18,attack prophet moham mean death kamlesh tiwar...,1,politics
43,wa manmohan singh lobbi for an individu dure ...,1,politics
45,attack prophet moham mean death kamlesh tiwar...,1,politics
120,wa manmohan singh lobbi for an individu dure ...,1,politics
...,...,...,...
1917,attack prophet moham mean death kamlesh tiwar...,1,politics
1992,wa manmohan singh lobbi for an individu dure ...,1,politics
1994,attack prophet moham mean death kamlesh tiwar...,1,politics
2019,wa manmohan singh lobbi for an individu dure ...,1,politics
